In [51]:
using Distributed

In [52]:
addprocs(4)

4-element Vector{Int64}:
 2
 3
 4
 5

In [56]:
@everywhere using LinearAlgebra

In [53]:
@everywhere function find_prime(vec)
	while true
		n = rand(vec)
		isprime(n) && return n
	end
end

In [54]:
function find_prime_with_timeout_mp(vec, timeout=0.5, target_pid=first(workers()))
    found_channel = Channel{Int}(1)
    @async begin
        value = remotecall_fetch(find_prime, target_pid, vec)
        isopen(found_channel) && put!(found_channel, value)
    end
    result = timedwait(timeout) do
        !isempty(found_channel)
    end
    @debug result
    close(found_channel)
    if result === :timed_out
        interrupt(target_pid)  # to stop finding a prime
        return  # `nothing`
    end
    fetch(found_channel)
end

find_prime_with_timeout_mp (generic function with 3 methods)

In [57]:
find_prime_with_timeout_mp(2:2:100_000_000_000, 0.2)

      From worker 2:	┌ Error: Fatal error on process 2
      From worker 2:	│   exception =
      From worker 2:	│    InterruptException:
      From worker 2:	│    Stacktrace:


Worker 2 terminated.


      From worker 2:	│     [1] serialize_msg(s::Distributed.ClusterSerializer{Sockets.TCPSocket}, o::Distributed.ResultMsg)
      From worker 2:	│       @ Distributed ~/.julia/juliaup/julia-1.9.2+0.aarch64.apple.darwin14/share/julia/stdlib/v1.9/Distributed/src/messages.jl:78
      From worker 2:	│     [2] #invokelatest#2
      From worker 2:	│       @ ./essentials.jl:816 [inlined]
      From worker 2:	│     [3] invokelatest
      From worker 2:	│       @ ./essentials.jl:813 [inlined]
      From worker 2:	│     [4] send_msg_(w::Distributed.Worker, header::Distributed.MsgHeader, msg::Distributed.ResultMsg, now::Bool)
      From worker 2:	│       @ Distributed ~/.julia/juliaup/julia-1.9.2+0.aarch64.apple.darwin14/share/julia/stdlib/v1.9/Distributed/src/messages.jl:181
      From worker 2:	│     [5] send_msg_now
      From worker 2:	│       @ ~/.julia/juliaup/julia-1.9.2+0.aarch64.apple.darwin14/share/julia/stdlib/v1.9/Distributed/src/messages.jl:118 [inlined]
      From worker 2:	│     [6

Unhandled Task ERROR: EOFError: read end of file
Stacktrace:
 [1] (::Base.var"#wait_locked#715")(s::Sockets.TCPSocket, buf::IOBuffer, nb::Int64)
   @ Base ./stream.jl:947
 [2] unsafe_read(s::Sockets.TCPSocket, p::Ptr{UInt8}, nb::UInt64)
   @ Base ./stream.jl:955
 [3] unsafe_read
   @ ./io.jl:761 [inlined]
 [4] unsafe_read(s::Sockets.TCPSocket, p::Base.RefValue{NTuple{4, Int64}}, n::Int64)
   @ Base ./io.jl:760
 [5] read!
   @ ./io.jl:762 [inlined]
 [6] deserialize_hdr_raw
   @ ~/.julia/juliaup/julia-1.9.2+0.aarch64.apple.darwin14/share/julia/stdlib/v1.9/Distributed/src/messages.jl:167 [inlined]
 [7] message_handler_loop(r_stream::Sockets.TCPSocket, w_stream::Sockets.TCPSocket, incoming::Bool)
   @ Distributed ~/.julia/juliaup/julia-1.9.2+0.aarch64.apple.darwin14/share/julia/stdlib/v1.9/Distributed/src/process_messages.jl:172
 [8] process_tcp_streams(r_stream::Sockets.TCPSocket, w_stream::Sockets.TCPSocket, incoming::Bool)
   @ Distributed ~/.julia/juliaup/julia-1.9.2+0.aarch64.apple.da

In [7]:
@everywhere function do_work(jobs, results)
    while true
        job_id = take!(jobs)
        exec_time = rand()
        sleep(exec_time) # 実際の仕事をする処理時間をシミュレートする。
        put!(results, (job_id, exec_time, myid()))
    end
end

In [3]:
@everywhere using LinearAlgebra

In [5]:
function test_sleep()
    @async begin
        for i in 1:10
            sleep(1)
            print("#")
        end
        return 10
    end
    sleep(2)
    
    return 2
end

test_sleep (generic function with 1 method)

In [6]:
test_sleep()

#

2

#########

In [71]:
function sleep6!(x::Vector{Int}, sw::Bool)
    for it in 1:6
        while(sw)
            sleep(1)
            print("#")
            push!(x, it)
        end
    end
end

sleep6! (generic function with 2 methods)

In [46]:
using Distributed

In [69]:
function test()
    x::Vector{Int} = []
    sw = true
    task1 = @task sleep6!(x, sw)
    
    schedule(task1)
    @async begin
        yield()
    end
    sleep(3)
    interrupt(workers())
    println(x)
    sw = false
    yield()
    return nothing
    #interrupt(workers())
end

test (generic function with 1 method)

In [70]:
test()

##[1, 2]


####

In [ ]:
function test2()
    c = Channel(0)
    x::Vector{Int} = []
    task1 = @task sleep6!(x)

In [41]:
function make_jobs(n)
    for i in 1:n
        put!(jobs, i)
    end
end

make_jobs (generic function with 1 method)

In [9]:
@async make_jobs(12)

Task (done) @0x0000000128740c90

In [10]:
@show jobs

jobs = RemoteChannel{Channel{Int64}}(1, 1, 22)


RemoteChannel{Channel{Int64}}(1, 1, 22)

In [11]:
for p in workers() # ワーカー上でタスクを開始して、リクエストを並列に処理する。
    remote_do(do_work, p, jobs, results)
end

In [13]:
n = 12

12

In [14]:
@elapsed while n > 0 # 結果を出力する
    job_id, exec_time, where = take!(results)
    println("$job_id finished in $(round(exec_time; digits=2)) seconds on worker $where")
    global n = n - 1
end

3 finished in 0.04 seconds on worker 2
4 finished in 0.14 seconds on worker 3
2 finished in 0.42 seconds on worker 4
1 finished in 0.86 seconds on worker 5
6 finished in 0.82 seconds on worker 3
5 finished in 0.94 seconds on worker 2
8 finished in 0.37 seconds on worker 5
7 finished in 0.97 seconds on worker 4
11 finished in 0.31 seconds on worker 5
10 finished in 0.48 seconds on worker 2
9 finished in 0.83 seconds on worker 3
12 finished in 0.69 seconds on worker 4


0.044327708

In [4]:
@everywhere function workers_work(N::Int)
    M = rand(Float32, N, N)
    M = Hermitian(M)
    e, v = eigen(M)
    return e
end

In [ ]:
@everywhere function check_timeout(Int::N, timeout=0.5)
    e = zeros(Float32, N)
    result = timedwait(timeout) do 
        e
    end